In [58]:
import torch
import os
import random
import cv2
import math
from torchvision import transforms
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary

In [2]:
os.makedirs('./train_output', exist_ok=True) # 创建文件夹

In [3]:
image_root = r'D:\learn\MNIST_Dataset\train_images'
txt = './train_output/total_train.txt'

In [4]:
os.listdir(image_root)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [5]:
sorted(os.listdir(image_root))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
os.path.isdir(os.path.join(image_root, '0'))

True

In [7]:
# 照片的名字以及对应的数字 写入一个txt文件中
def image_list(imageRoot, txt):
    f = open(txt, 'wt')
    for (label, filename) in enumerate(sorted(os.listdir(image_root), reverse = False)):
        print('label = {}, filename = {}'.format(label, filename))
        if os.path.isdir(os.path.join(image_root, filename)):
            for imagename in os.listdir(os.path.join(image_root, filename)):
                name, ext = os.path.splitext(imagename)
                ext = ext[1:]
                if ext == 'jpg' or ext == 'png' or ext == 'bmp':
                    f.write('%s %d\n' % (os.path.join(imageRoot, filename, imagename), label))
    f.close()
image_list(imageRoot = image_root, txt = txt)

label = 0, filename = 0
label = 1, filename = 1
label = 2, filename = 2
label = 3, filename = 3
label = 4, filename = 4
label = 5, filename = 5
label = 6, filename = 6
label = 7, filename = 7
label = 8, filename = 8
label = 9, filename = 9


In [8]:
trainFile = r'D:\learn\下载git的代码\mnist\train_output\train.txt'
valFile = r'D:\learn\下载git的代码\mnist\train_output\val.txt'
def shuffle_split(listFile, trainFile, valFile):
    with open(listFile, 'r') as f:
        records = f.readlines()
    random.shuffle(records)
    num = len(records)
    trainNum = int(num * 0.8) # 训练集占80%
    with open(trainFile, 'w') as f:
        f.writelines(records[:trainNum])
    with open(valFile, 'w') as f1:
        f1.writelines(records[trainNum:])
shuffle_split(listFile = txt, trainFile = trainFile, valFile = valFile)

In [9]:
test = []
fh = open(trainFile, 'r')
for line in fh:
    #print(line)
    # strip : 移除字符串头尾指定的字符（默认为空格或换行符）或字符序列
    line = line.strip('\n') # ['D:\\learn\\MNIST_Dataset\\train_images\\4\\4_04811.jpg 4', '']
    #print(line)
    line = line.rstrip() # # 删除 string 字符串末尾的指定字符，默认为空白符，包括空格、换行符、回车符、制表符。
    words = line.split()
    test.append((words[0], int(words[1])))

In [10]:
class MyDataset(Dataset):
    def __init__(self, txt, transform = None, target_transform = None):
        fh = open(txt, 'r')
        imgs = []
        for line in fh:  # line为每一行
            line = line.strip('\n')
            line = line.rstrip() # 删除 string 字符串末尾的指定字符，默认为空白符，包括空格、换行符、回车符、制表符。
            words = line.split()
            imgs.append((words[0], int(words[1])))
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        fn ,label = self.imgs[index]
        img = cv2.imread(fn, cv2.IMREAD_COLOR)
        if self.transform is not None:
            img = self.transform(img)
        return img, label
    
    def __len__(self):
        return len(self.imgs)

In [11]:
train_data = MyDataset(txt = trainFile, transform = transforms.ToTensor())

In [12]:
val_data = MyDataset(txt = valFile, transform = transforms.ToTensor())

In [14]:
batch_size = 128
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(dataset = val_data, batch_size = batch_size)

In [50]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 1), # 32 * 28 *28
            nn.ReLU(),
            nn.MaxPool2d(2) # 32 * 14 * 14
        )
        self. conv2 = nn.Sequential(
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1), # 64 * 14 *14
            nn.ReLU(),
            nn.MaxPool2d(2) # 64 x 7 x 7
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2) # 64 x 3 x3
        )

        self.dense = nn.Sequential(
            nn.Linear(64 * 3 * 3, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        res = conv3_out.reshape(conv3_out.size(0), -1)
        out = self.dense(res)
        return out
model = Net()

In [51]:
print(model)

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=576, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [52]:
summary(model, (3, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             896
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7             [-1, 64, 7, 7]          36,928
              ReLU-8             [-1, 64, 7, 7]               0
         MaxPool2d-9             [-1, 64, 3, 3]               0
           Linear-10                  [-1, 128]          73,856
             ReLU-11                  [-1, 128]               0
           Linear-12                   [-1, 10]           1,290
Total params: 131,466
Trainable params: 131,466
Non-trainable params: 0
-------------------------------

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 1e-3)
# 在第10轮的时候lr * 0.1， 在第20轮后 (lr * 0.1) * 0.1
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [10, 20], 0.1)
loss_func = nn.CrossEntropyLoss()

In [56]:
scheduler

In [60]:
epochs = 30
epoch_list = []
for epoch in range(epochs):
    epoch_list.append(epoch)
    # train--------------------------
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for batch, (batch_x, batch_y) in enumerate(train_loader):
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        train_loss += loss.item()
        pred = torch.max(out, 1)[1]
        train_correct = (pred == batch_y).sum()
        train_acc += train_correct.item()
        print('epoch: %2d/%d batch %3d/%d  Train Loss: %.3f, Acc: %.3f'
        % (epoch + 1, epochs, batch, math.ceil(len(train_data) / batch_size),
            loss.item(), train_correct.item() / len(batch_x)))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step() # 更新lr
    print('Train Loss: %.6f, Acc: %.3f' % (train_loss / (math.ceil(len(train_data)/batch_size)),
                                        train_acc / (len(train_data))))
    
    # evaluation-----------
    model.eval()
    eval_loss = 0.0
    eval_acc = 0.0
    for batch_x, batch_y in val_loader:
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        eval_loss += loss.item()
        pred = torch.max(out, 1)[1]
        num_correct = (pred == batch_y).sum()
        eval_acc += num_correct.item()
    print('Val Loss: %.6f, Acc: %.3f' % (eval_loss / (math.ceil(len(val_data)/batch_size)),
                                            eval_acc / (len(val_data))))
    
    # save_model
    if (epoch + 1) % 1 == 0:
        torch.save(model.state_dict(), 'D:/learn/mnist_model/params_' + str(epoch + 1) + '.pth')


epoch:  1/30 batch   0/375  Train Loss: 0.047, Acc: 0.984
epoch:  1/30 batch   1/375  Train Loss: 0.074, Acc: 0.969
epoch:  1/30 batch   2/375  Train Loss: 0.047, Acc: 0.984
epoch:  1/30 batch   3/375  Train Loss: 0.074, Acc: 0.984
epoch:  1/30 batch   4/375  Train Loss: 0.081, Acc: 0.977
epoch:  1/30 batch   5/375  Train Loss: 0.064, Acc: 0.977
epoch:  1/30 batch   6/375  Train Loss: 0.093, Acc: 0.969
epoch:  1/30 batch   7/375  Train Loss: 0.085, Acc: 0.984
epoch:  1/30 batch   8/375  Train Loss: 0.043, Acc: 0.977
epoch:  1/30 batch   9/375  Train Loss: 0.073, Acc: 0.969
epoch:  1/30 batch  10/375  Train Loss: 0.065, Acc: 0.977
epoch:  1/30 batch  11/375  Train Loss: 0.054, Acc: 0.984
epoch:  1/30 batch  12/375  Train Loss: 0.098, Acc: 0.977
epoch:  1/30 batch  13/375  Train Loss: 0.069, Acc: 0.984
epoch:  1/30 batch  14/375  Train Loss: 0.130, Acc: 0.961
epoch:  1/30 batch  15/375  Train Loss: 0.054, Acc: 0.984
epoch:  1/30 batch  16/375  Train Loss: 0.040, Acc: 0.984
epoch:  1/30 b